In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import time
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
def is_responder(data):
    return data['Score'] > 50

In [3]:
def get_neighbours(lines, patient, neighbours_count):
    lines_features = lines.iloc[:, 1:]
    lines_score = 2 * is_responder(lines) - 1
    
    patient_features = patient.iloc[1:]
        
    diff = np.array(lines_features - patient_features).astype(float)
    distances = np.linalg.norm(diff, axis=1)
        
    idx = np.argpartition(distances, neighbours_count)
    neighbours_lines_features = lines_features.iloc[idx[:neighbours_count], :]
    neighbours_lines_score = lines_score.iloc[idx[:neighbours_count]]
    
    return neighbours_lines_features, neighbours_lines_score

In [4]:
def get_division_value(model, patients):
    patients_score = 2 * is_responder(patients) - 1
    patients_values = []
    
    for _, patient in patients.iterrows():
        patients_values.append(model.predict(patient.iloc[1:].reshape(1,-1)))
    
    best_division_value = 1e100
    best_quality = -1

    division_values = []
    for value in patients_values:
        division_values.append(value + 1e-10)

    for division_value in division_values:
        n_right_classified = 0
        for idx in range(len(patients_values)):
            if (patients_values[idx] - division_value) * patients_score[idx] > 0:
                n_right_classified += 1
        _quality = n_right_classified / len(patients_values)
        if _quality > best_quality:
            best_quality = _quality
            best_division_value = division_value
        
    return best_division_value

In [5]:
def check_regression(model, patients):
    patients_features = patients.iloc[:, 1:]
    patients_score = 2 * is_responder(patients) - 1
    
    regression_quality = 0
    
    patients_values = []
    
    for _, patient in patients.iterrows():
        patients_values.append(model.predict(patient.iloc[1:].reshape(1,-1)))
    
    patients_values = np.array(patients_values)
    
    if patients.shape[0] > 50:
        best_division_value = get_division_value(model, patients)
    
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(patients_features):
        test_patients_values = patients_values[test_index]
        test_patients_score = patients_score[test_index]
        
        if patients.shape[0] <= 50:
            best_division_value = get_division_value(model, patients.iloc[train_index])
        
        pred_reg_class = float(test_patients_values) > float(best_division_value)
        
        true_reg_class = float(test_patients_score) > 0
        
        regression_quality += (pred_reg_class == true_reg_class) / len(patients_values)
    
    return regression_quality

In [6]:
class Classifier(object):
    
    def __init__(self, quality_trigger, stability_trigger, norm, neighbours_count):
        self.nc = neighbours_count
        self.model = None
        self.ready = False
        self.quality_trigger = quality_trigger
        self.stability_trigger = stability_trigger
        self.norm = norm
    
    def fit(self, lines, patient, model_type='ridge'):
        self.positive = is_responder(patient)
        
        if model_type == 'linear':
            model = LinearRegression(fit_intercept=True, normalize=self.norm)
        if model_type == 'lasso':
            model = Lasso(fit_intercept=True, normalize=self.norm)
        if model_type == 'ridge':
            model = Ridge(fit_intercept=True, normalize=self.norm)
        
        self.model_type = model_type
        
        neighbours_lines_features, neighbours_lines_score = get_neighbours(lines, patient, self.nc)
        
        model.fit(neighbours_lines_features, neighbours_lines_score)
        
        # regression quality assessment
        predictions, errors = [], []

        loo = LeaveOneOut()
        for train_index, test_index in loo.split(neighbours_lines_features):
            x_train = neighbours_lines_features.iloc[train_index]
            y_train = neighbours_lines_score.iloc[train_index]
            x_test = neighbours_lines_features.iloc[test_index]
            y_test = neighbours_lines_score.iloc[test_index]

            model.fit(x_train, y_train)

            prediction = model.predict(x_test)

            predictions.append(prediction)

            errors.append(prediction - y_test)

        sigma = np.sqrt(np.mean(np.array(errors) ** 2))

        m = np.mean(predictions)

        self.stability = sigma / m
        
        if self.stability <= self.stability_trigger:
            self.ready = True
            self.model = model

    def predict_reg(self, patients_features):
        if self.ready:
            return self.model.predict(np.array(patients_features).reshape(-1, patients_features.shape[1]))
        
    def __str__(self):
        return str([self.model_type, self.nc, self.stability])

In [7]:
class Ensemble(object):
    
    def __init__(self, quality_trigger=0.7, norm=False, stability_trigger=0.1):
        self.classifiers = []
        self.quality_trigger = quality_trigger
        self.stability_trigger = stability_trigger
        self.norm = norm
        
    def fit_kfold(self, lines, patients, patients_count=None, nc_list=[50,100,150], random_state=0):
        if patients_count is None:
            patients_count = patients.shape[0]
        
        print("Quality border: {0}".format(self.quality_trigger))
        print("Stability border: {0}".format(self.stability_trigger))
        
        # construction and stability check
        all_clfs = []
        for _, patient in patients.iterrows():
            for neighbours_count in nc_list:
                for model_type in ['ridge']:
                    
                    classifier = Classifier(self.quality_trigger, self.stability_trigger,
                                            self.norm, neighbours_count)
                    classifier.fit(lines, patient, model_type)
                    
                    if classifier.ready:
                        all_clfs.append(classifier)
        
        print("Constructed {0} stable regressions".format(len(all_clfs)))
        
        # information check
        kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)
        
        good_clfs = []
        
        fold = 1
        for train_index, test_index in kfold.split(patients):
            print("Fold {0}".format(fold))
            train_data = patients.iloc[train_index]
            
            for clf in all_clfs:
                clf_quality = check_regression(clf.model, train_data)
                if clf_quality >= self.quality_trigger:
                    good_clfs.append(clf)
            
            print("Keep {0} regressions".format(len(good_clfs)))
            all_clfs, good_clfs = good_clfs, []
            fold += 1
        
        self.classifiers = all_clfs
        
    def assess_quality_kfold(self, patients, random_state=0):
        start_time = time.time()
        
        cv_acc, cv_pos, cv_neg = [], [], []

        clf_acc = np.zeros((patients.shape[0], len(self.classifiers)))
        
        clf_fold_results = np.zeros((5, len(self.classifiers)))
        
        kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

        lap = 0

        for train_index, test_index in kfold.split(patients):
            lap += 1
            print("\nLap {0}".format(lap))
            
            test_acc, test_pos, test_neg = [], [], []

            for _id_ in test_index:

                predictions = []

                x_test = patients_data.iloc[_id_:_id_+1, 1:]

                true_class = is_responder(patients_data.iloc[_id_])

                for clf_id in range(len(self.classifiers)):
                    clf = self.classifiers[clf_id]

                    current_division_value = get_division_value(clf.model, patients_data.iloc[train_index])

                    pred_class = clf.predict_reg(x_test) > current_division_value

                    clf_acc[_id_, clf_id] = (pred_class == true_class)

                    predictions.append(pred_class == true_class)

                _res_ = np.sum(predictions) / len(predictions)
                
                _res_ = _res_ > 0.5
                
                test_acc.append(_res_)

                if true_class:
                    test_pos.append(_res_)
                else:
                    test_neg.append(_res_)

            cv_acc.append(np.mean(test_acc))
            cv_pos.append(np.mean(test_pos))
            cv_neg.append(np.mean(test_neg))

            print("Ensemble accuracy: {0:.3f}".format(np.mean(test_acc)))

            print("Single clf results")
            fold_clf_results = np.round(clf_acc[test_index,:].mean(axis=0), decimals=3)
            clf_fold_results[lap - 1,:] = fold_clf_results
            print(fold_clf_results)

        print("")
        print(np.round(cv_acc, decimals=3))
        print(np.round(cv_pos, decimals=3))
        print(np.round(cv_neg, decimals=3))
        print("")

        print("Total accuracy: {0:.3f}".format(np.mean(cv_acc)))
        print("Positive accuracy: {0:.3f}".format(np.mean(cv_pos)))
        print("Negative accuracy: {0:.3f}".format(np.mean(cv_neg)))
        
        print("\n {0} sec".format(time.time() - start_time))
        
        return clf_fold_results
    
    def print(self, full=False):
        print(len(self.classifiers), "regressions")
        if full:
            for clf in self.classifiers:
                print("regression type: {0} \t neighbours count: {1}".format(clf.model_type, clf.nc))